In [3]:
import pandas as pd
import numpy as np

In [4]:
companies_selected = pd.read_excel("Companies.xlsx")

In [5]:
# since TSR is in % we will grade wight in % as well
grade_weight = {"A": 90, "B": 80, "C": 70, "D": 60, "E": 50, "F": 40}
companies_selected["GradeWeight"] = companies_selected["Grade"].map(grade_weight)

In [6]:
def calculate_grade_score(data, x):
    return (1 - x) * data["TSR"] + x * data["GradeWeight"]

In [22]:
# default values set as: 16% for industry allocation and 4.5% mac allocation
# beta value for potfolio should be below 1
def monte_carlo_simulation(
    data, max_industry_allocation=0.16, max_stock_allocation=0.045, target_beta=1.0
):

    best_portfolio = None
    best_portfolio_beta = float("inf")

    for i in range(5000, 8000):
        i = i / 10000
        # we first want to normalise the TSR:
        data["Industry TSR total"] = data.groupby("Industry")["TSR"].transform("sum")
        data["weighted TSR"] = data["TSR"] / data["Industry TSR total"]
        # This will give us cases where our grade is not the main factor and TSR is which provides us with different scores on multiple iterations
        # execute the function mention above
        data["Grade Score"] = data.apply(calculate_grade_score, axis=1, args=(i,))

        # obtain a weight scross all the investments. Basically getting a ratio so values for the weights will be between 0 and 1
        # basic normalisation
        data["Weight"] = data["Grade Score"] / data["Grade Score"].sum()

        # Lets focus on each industry
        # what is the current weight allocated to each industry
        data["Industry Allocation"] = data.groupby("Industry")["Weight"].transform(
            "sum"
        )
        data["Allocation_by_restriction"] = np.where(
            data["Industry Allocation"] > max_industry_allocation,
            max_industry_allocation,
            data["Industry Allocation"],
        )

        data["Normalised Weight"] = np.where(
            data["Allocation_by_restriction"] == 0.16,
            (data["Weight"] / data["Industry Allocation"]) * 0.16,
            data["Weight"],
        )

        # Calculate Portfolio Beta
        portfolio_beta = (data["Beta Value"] * data["Normalised Weight"]).sum()

        if portfolio_beta < best_portfolio_beta:
            best_run = i
            target_beta = portfolio_beta
            best_portfolio_investment = data
            print(best_run, target_beta)

    return best_run, target_beta, best_portfolio_investment

In [ ]:
# FIGURE OUT:
# - As x increases, the objective functions get skewed to give more weightage to the 'GRADE WEIGHT' term
# - Also as x increases, the final beta value also keeps increasing (check output in the cell below), WHY?

In [24]:
run_no, final_beta, ideal_portfolio = monte_carlo_simulation(companies_selected)

0.5 0.8383853124598657
0.5001 0.8384111509727522
0.5002 0.838436991641182
0.5003 0.8384628344653667
0.5004 0.8384886794455167
0.5005 0.8385145265818433
0.5006 0.8385403758745577
0.5007 0.8385662273238708
0.5008 0.838592080929993
0.5009 0.8386179366931363
0.501 0.8386437946135108
0.5011 0.8386696546913277
0.5012 0.8386955169267983
0.5013 0.838721381320133
0.5014 0.838747247871543
0.5015 0.8387731165812389
0.5016 0.8387989874494316
0.5017 0.8388248604763329
0.5018 0.8388507356621521
0.5019 0.8388766130071013
0.502 0.838902492511391
0.5021 0.8389283741752319
0.5022 0.8389542579988349
0.5023 0.8389801439824108
0.5024 0.8390060321261705
0.5025 0.8390319224303247
0.5026 0.8390578148950842
0.5027 0.8390837095206596
0.5028 0.8391096063072621
0.5029 0.8391355052551019
0.503 0.8391614063643901
0.5031 0.8391873096353372
0.5032 0.8392132150681544
0.5033 0.8392391226630518
0.5034 0.8392650324202403
0.5035 0.8392909443399307
0.5036 0.8393168584223335
0.5037 0.8393427746676593
0.5038 0.83936869307611

In [17]:
run_no

0.7999

In [19]:
final_beta

0.9263227009210285

In [106]:
ideal_portfolio

,Company Name,Industry,Beta Value,Grade,TSR,GradeWeight,Industry TSR total,weighted TSR,Grade Score,Weight,Industry Allocation,Allocation_by_restriction,Normalised Weight
0,NextEra Energy (NEE),Utilities,0.49,B,53.47,80,358.64,0.149091,74.691347,0.010767,0.103745,0.103745,0.010767
1,Duke Energy (DUK),Utilities,0.38,C,57.51,70,358.64,0.160356,67.500751,0.009731,0.103745,0.103745,0.009731
2,Atmos Energy (ATO),Utilities,0.50,B,54.13,80,358.64,0.150931,74.823413,0.010786,0.103745,0.103745,0.010786
3,Avangrid (AGR),Utilities,0.34,A,-8.57,90,358.64,-0.023896,70.276143,0.010131,0.103745,0.103745,0.010131
4,Otter Tail (OTTR),Utilities,0.69,A,87.72,90,358.64,0.244591,89.543772,0.012908,0.103745,0.103745,0.012908
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,COSTCO WHOLESALE CORPORATION (XNAS:COST),Consumer Staples,0.79,D,17.43,60,177.69,0.098092,51.481743,0.007421,0.097373,0.097373,0.007421
75,DOLLAR GENERAL CORPORATION (XNYS:DG),Consumer Staples,0.45,B,14.09,80,177.69,0.079295,66.811409,0.009631,0.097373,0.097373,0.009631
76,COLGATE-PALMOLIVE COMPANY (XNYS:CL),Consumer Staples,0.41,A,15.40,90,177.69,0.086668,75.072540,0.010822,0.097373,0.097373,0.010822
77,THE HERSHEY COMPANY (XNYS:HSY),Consumer Staples,0.37,B,12.95,80,177.69,0.072880,66.583295,0.009598,0.097373,0.097373,0.009598
